In [1]:
import pandas as pd
import altair as alt
import time

In [2]:
traffic = (pd.read_excel("data/traffic/cleaned/aggregated-combined-traffic-data.xlsx")).fillna(0)

In [3]:
# studying dataframe, 2017 & 2018 data unavailable for inter #6: 
traffic

,Intersection,Time,YR_2016,YR_2017,YR_2018,Total
0,6,01:00 AM,4.571429,0.00,0.0,4.571429
1,6,01:00 PM,57.142857,0.00,0.0,57.142857
2,6,02:00 AM,2.928571,0.00,0.0,2.928571
3,6,02:00 PM,60.642857,0.00,0.0,60.642857
4,6,03:00 AM,1.714286,0.00,0.0,1.714286
...,...,...,...,...,...,...
91,1071,10:00 PM,5.333333,3.00,4.5,4.428571
92,1071,11:00 AM,5.000000,5.75,8.0,6.071429
93,1071,11:00 PM,2.833333,1.75,1.5,2.142857
94,1071,12:00 AM,2.666667,2.75,1.0,2.214286


In [4]:
# rename cols
traffic = traffic.rename(columns={'YR_2016':'2016', 'YR_2017': "2017", 'YR_2018':'2018'})

In [5]:
# transform data into long form
traf = traffic.melt(id_vars=["Intersection", "Time"], value_vars = ["2016","2017","2018"], var_name="Year", 
                    value_name="Traffic")

traf

,Intersection,Time,Year,Traffic
0,6,01:00 AM,2016,4.571429
1,6,01:00 PM,2016,57.142857
2,6,02:00 AM,2016,2.928571
3,6,02:00 PM,2016,60.642857
4,6,03:00 AM,2016,1.714286
...,...,...,...,...
283,1071,10:00 PM,2018,4.500000
284,1071,11:00 AM,2018,8.000000
285,1071,11:00 PM,2018,1.500000
286,1071,12:00 AM,2018,1.000000


In [6]:
# create column for hour of day
traf['Hour'] = traf['Time'].str.split(':').str[0].astype(int)

# 12 AM is 0
traf.loc[traf['Time'].str.contains('12:'), 'Hour'] = 0

# add 12 if PM
traf.loc[traf['Time'].str.contains('PM'), 'Hour'] = traf['Hour'] + 12

# show dataframe
traf

,Intersection,Time,Year,Traffic,Hour
0,6,01:00 AM,2016,4.571429,1
1,6,01:00 PM,2016,57.142857,13
2,6,02:00 AM,2016,2.928571,2
3,6,02:00 PM,2016,60.642857,14
4,6,03:00 AM,2016,1.714286,3
...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,22
284,1071,11:00 AM,2018,8.000000,11
285,1071,11:00 PM,2018,1.500000,23
286,1071,12:00 AM,2018,1.000000,0


In [7]:
# reorder times for slider
times = traf["Time"].to_list()
am, pm = [], []
for tme in times[:23]:
    ls = (tme.split(" "))
    t = ls[1]
    if t == "PM":
        tme = time.strptime(tme.strip(), "%I:%M %p")
        pm.append(tme) 
    elif tme == "12:00 AM":
        tme = time.strptime(tme.strip(), "%I:%M %p")
        am.insert(0, tme)
    else:
        tme = time.strptime(tme.strip(), "%I:%M %p")
        am.append(tme)

times = am + pm

In [8]:
from datetime import datetime, timedelta

from bokeh.models import CustomJS, DateRangeSlider
from bokeh.plotting import show

start = datetime.now()
end = start + timedelta(days=1)
slider = DateRangeSlider(value=(start, end), start=start, end=end, format="%x, %X")

# show(slider)

# https://docs.bokeh.org/en/latest/docs/gallery/slider.html
# https://stackoverflow.com/questions/60332328/time-range-slider-for-bokeh

In [9]:
'''# brush notation and slider notation, incomplete
slider = alt.binding_range(min=traf.Time.min(), max=traf.Time.min(), 
                           step=1)
select_time = alt.selection_single(name='Time', fields=['Time'],
                                   bind=slider)'''


"# brush notation and slider notation, incomplete\nslider = alt.binding_range(min=traf.Time.min(), max=traf.Time.min(), \n                           step=1)\nselect_time = alt.selection_single(name='Time', fields=['Time'],\n                                   bind=slider)"

In [10]:
alt.Chart(traf).mark_bar().encode(
    x='Traffic:Q', # trying to plot it to see if tf worked
    y='Year:N',
    color=alt.Color('Year', scale=alt.Scale(scheme='dark2'))
).properties(height=400, width=600)

/Users/sree/anaconda3/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)